# A) pre-processing

## 0. import packages

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from openpyxl import load_workbook
import translators as ts
import pycountry

Using Slovakia server backend.


## 0A. functions

In [25]:
# remove not translatable colors
def changeStr(rrr):
    if "mét." in rrr:
        return rrr[:-5]
    else:
        return rrr

# simplify body type name
def changeBody(b):
    if " / " in b:
        return b.split(" / ")[0]
    else:
        return b

In [2]:
# remove pandas formatting from excel
def removeFormatting(ws):
    # ws is not the worksheet name, but the worksheet object
    for row in ws.iter_rows():
        for cell in row:
            cell.style = 'Normal'

In [28]:
# translate to english german content
def getEnglish(lst):
    di = {'Name': lst}
    vls = list()
    try:
        for v in di.values():
            for i in v:
                if i != "null":
                    vls.append(ts.google(i, from_language='de', to_language='en'))
                else:
                    vls.append('Other')
        di2 = {'Values':vls}
        di.update(di2)
        return di
    except:
        return di

In [29]:
# obtain country code based on city name
def getCountry(lst):
    di = {'Name': lst}
    vls = list()
    geolocator = Nominatim(user_agent="MyApp")
    try:
        for v in di.values():
            for i in v:
                if i != "null":
                    location = geolocator.geocode(i, language="en") 
                    ctyname = list(location.address.split(", "))[-1]
                    ctycode = pycountry.countries.get(name=ctyname).alpha_2
                    vls.append(ctycode)
                else:
                    vls.append('Other')
        di2 = {'Values':vls}
        di.update(di2)
        return di
    except:
        return di

In [38]:
# derive mileage consumption unit
def consUnit(txt):
    if 'l/100km' in txt:
        return 'l_km_consumption'
    else:
        return 'null'

# derive mileage unit    
def mil(txt):
    if 'km' in txt:
        return 'kilometer'
    else:
        return 'mile'
# derive Left Hand Drive - LHD
# or Right Hand Drive from Country Code
def drive(txt):
    if 'GB' in txt:
        return 'RHD'
    else:
        return 'LHD'

# format mileage in the required format
def mileage_conv(val):
    val = int(val)
    return '{0:.2f}'.format(val)

## 1. import data

In [3]:
raw = pd.read_json('rawdata/supplier_car.json', lines=True)
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21906 entries, 0 to 21905
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                21906 non-null  int64 
 1   MakeText          21906 non-null  object
 2   TypeName          21906 non-null  object
 3   TypeNameFull      21906 non-null  object
 4   ModelText         20957 non-null  object
 5   ModelTypeText     21906 non-null  object
 6   Attribute Names   21906 non-null  object
 7   Attribute Values  21906 non-null  object
 8   entity_id         21906 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.5+ MB


## 2. data exploration

In [4]:
raw.head(5)

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
0,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
1,1059,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,Hp,235,00107c2d-0071-4475-88f0-810133638b7e
2,524,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,FuelTypeText,Benzin,00126794-a8ef-48fe-93d6-43cfc69fbfb6
3,608,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,Ccm,1985,00182529-1bf7-4f93-89fa-2e8e634b2c9d
4,726,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,BodyColorText,schwarz mét.,002d30c2-43f6-4905-868f-160dbc445c56


In [5]:
raw.isna().sum()

ID                    0
MakeText              0
TypeName              0
TypeNameFull          0
ModelText           949
ModelTypeText         0
Attribute Names       0
Attribute Values      0
entity_id             0
dtype: int64

In [6]:
raw['Attribute Names'].unique()

array(['Seats', 'Hp', 'FuelTypeText', 'Ccm', 'BodyColorText',
       'ConsumptionTotalText', 'Doors', 'Co2EmissionText',
       'TransmissionTypeText', 'FirstRegYear', 'Properties',
       'ConditionTypeText', 'Km', 'DriveTypeText', 'InteriorColorText',
       'ConsumptionRatingText', 'FirstRegMonth', 'City', 'BodyTypeText'],
      dtype=object)

check if entity_id is unique or shared among different attributes

In [7]:
raw[raw['entity_id']=='0001fda6-192b-46a8-bc08-0e833f904eed']

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
0,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed


check if ModelTypeText or ID determine a sufficient group of attributes

In [8]:
raw[raw.ModelTypeText=='SLR McLaren'].shape

(19, 9)

In [9]:
raw[raw.ID==976].shape

(19, 9)

In [10]:
raw[raw.ID==976]

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
0,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
93,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Properties,"""Ab MFK""",10f256ec-800b-44d1-8645-a80cc3ba227f
677,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,FuelTypeText,Benzin,843da678-76ec-4a0b-98c1-670653316199
1203,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Hp,626,e8130e4a-6d0e-4e7c-8570-3cb91102b7e2
1763,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,City,Zuzwil,4d828910-b1a1-464b-a703-411b5b302494
4303,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,FirstRegYear,2007,2c3bfb1a-d9fe-474d-9c35-7ac80f918522
4585,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Doors,2,5c8f7e88-fcba-48d1-9aee-ef0e01fd6f9f
6229,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,ConsumptionTotalText,null,89df4495-0d66-4adc-b352-544d90afb6ac
7522,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,ConditionTypeText,Occasion,7dce606c-a20f-438f-92d9-235519d0f383
11326,976,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,FirstRegMonth,10,50d801aa-ab99-49ff-8c75-c69864dc22ae


## 3. data preparation

In [11]:
test = raw.pivot(index='ID', columns='Attribute Names', values='Attribute Values').reset_index()

In [12]:
test.shape

(1153, 20)

In [13]:
test[test.ID == 976]

Attribute Names,ID,BodyColorText,BodyTypeText,Ccm,City,Co2EmissionText,ConditionTypeText,ConsumptionRatingText,ConsumptionTotalText,Doors,DriveTypeText,FirstRegMonth,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText
975,976,silber mét.,Cabriolet,5439,Zuzwil,null,Occasion,null,null,2,Hinterradantrieb,10,2007,Benzin,626,schwarz,29800,"""Ab MFK""",2,Automatik-Getriebe


_joining new dataframe with reworked main raw data_

In [14]:
raw_main = raw.copy()

In [15]:
rawu = raw_main[['ID','MakeText','TypeName','TypeNameFull','ModelText','ModelTypeText']].drop_duplicates()

In [16]:
rawu.shape

(1153, 6)

_since the number of rows is equal we can safely join_

In [17]:
final = pd.merge(rawu,test,how='inner',on='ID')

In [52]:
final.shape

(1153, 25)

In [18]:
final.isna().sum()

ID                        0
MakeText                  0
TypeName                  0
TypeNameFull              0
ModelText                50
ModelTypeText             0
BodyColorText             0
BodyTypeText              1
Ccm                       0
City                      0
Co2EmissionText           0
ConditionTypeText         0
ConsumptionRatingText     0
ConsumptionTotalText      0
Doors                     0
DriveTypeText             0
FirstRegMonth             0
FirstRegYear              0
FuelTypeText              0
Hp                        0
InteriorColorText         0
Km                        0
Properties                0
Seats                     0
TransmissionTypeText      0
dtype: int64

_export preprocessing data to excel_

In [19]:
ex = final.sort_values('ID').reset_index(drop=True)

In [20]:
ex = ex[
    [
        'MakeText',
        'ModelText',
        'ModelTypeText',
        'BodyColorText',
        'BodyTypeText',
        'City',
        'ConditionTypeText',
        'ConsumptionTotalText',
#         'Doors',
#         'DriveTypeText',
        'FirstRegMonth',
        'FirstRegYear',
        'Km'
        # 'Properties'
        # 'Seats'
    ]
]

In [21]:
ex.shape

(1153, 11)

In [22]:
ex.to_excel('rawdata/pre_processed_data2.xlsx', index=False)

# B) normalisation

_making a copy of the pre-processed dataframe_

In [23]:
norm = ex.copy()

_filling null values with Other_

In [24]:
norm = norm.fillna('Other')

_format Body Color and Body Type_

In [26]:
norm.BodyColorText = norm.BodyColorText.apply(changeStr)
norm.BodyTypeText = norm.BodyTypeText.apply(changeBody)

_obtain list of unique values for colors, body types, condition and cities_

In [27]:
colors = list(norm.BodyColorText.drop_duplicates())
colors = [ x for x in colors if "Mét." not in x ]

# drive = list(norm.DriveTypeText.drop_duplicates())

body = list(norm.BodyTypeText.drop_duplicates().dropna())

condition = list(norm.ConditionTypeText.drop_duplicates())

cities = list(norm.City.drop_duplicates())

_leveraging external APIs to obtain data for missing features in input data_

In [30]:
bodies = pd.DataFrame.from_dict(getEnglish(body))
conditions = pd.DataFrame.from_dict(getEnglish(condition))
cols = pd.DataFrame.from_dict(getEnglish(colors))
cts = pd.DataFrame.from_dict(getCountry(cities))

_enriching normalized dataframe with new features_

In [33]:
norm = pd.merge(norm, conditions, how='left', left_on=['ConditionTypeText'], right_on=['Name'])
norm.drop(columns={'Name','ConditionTypeText'}, inplace=True)
norm.rename(columns={'Values':'condition'}, inplace=True)

In [34]:
norm = pd.merge(norm, bodies, how='left', left_on=['BodyTypeText'], right_on=['Name'])
norm.drop(columns={'Name','BodyTypeText'}, inplace=True)
norm.rename(columns={'Values':'carType'}, inplace=True)

In [36]:
norm = pd.merge(norm, cols, how='left', left_on=['BodyColorText'], right_on=['Name'])
norm.drop(columns={'Name','BodyColorText'}, inplace=True)
norm.rename(columns={'Values':'color'}, inplace=True)

In [37]:
norm = pd.merge(norm, cts, how='left', left_on=['City'], right_on=['Name'])
norm.drop(columns={'Name'}, inplace=True)
norm.rename(columns={'Values':'country'}, inplace=True)

In [39]:
norm['fuel_consumption_unit'] = norm.ConsumptionTotalText.apply(consUnit)
norm['mileage_unit'] = norm.ConsumptionTotalText.apply(mil)
norm.drop(columns={'ConsumptionTotalText'},inplace=True)

In [40]:
norm['drive'] = norm.country.apply(drive)

In [53]:
norm.shape

(1153, 14)

In [41]:
norm.head(3)

,MakeText,ModelText,ModelTypeText,City,FirstRegMonth,FirstRegYear,Km,condition,carType,color,country,fuel_consumption_unit,mileage_unit,drive
0,MERCEDES-BENZ,E 320,E 320 Elégance 4-Matic,Zuzwil,1,1999,31900,Occasion,limousine,anthracite,CH,l_km_consumption,kilometer,LHD
1,AUDI,RS6,RS6 Avant 5.0 V10 quattro,Zuzwil,7,2008,25400,Occasion,Combi,anthracite,CH,l_km_consumption,kilometer,LHD
2,AUDI,RS6,RS6 Avant quattro,Zuzwil,10,2002,38500,Occasion,Combi,anthracite,CH,l_km_consumption,kilometer,LHD


# C) integration

_making a copy of the first iteration of the normalized dataframe_

In [42]:
norm2 = norm.copy()

_renaming column to fit target data_

In [43]:
norm2.rename(columns={
    'MakeText':'make',
    'ModelText':'model',
    'ModelTypeText':'model_variant',
    'City':'city',
    'FirstRegMonth':'manufacture_month',
    'FirstRegYear':'manufacture_year',
    'Km':'mileage'
},inplace=True)

_adding final missing features_

In [44]:
norm2['currency'] = "CHF"
norm2['price_on_request'] = "false"
norm2['type'] = "car"
norm2['zip'] = "null"

In [45]:
print(norm2.shape)
norm2.head(3)

(1153, 18)


,make,model,model_variant,city,manufacture_month,manufacture_year,mileage,condition,carType,color,country,fuel_consumption_unit,mileage_unit,drive,currency,price_on_request,type,zip
0,MERCEDES-BENZ,E 320,E 320 Elégance 4-Matic,Zuzwil,1,1999,31900,Occasion,limousine,anthracite,CH,l_km_consumption,kilometer,LHD,CHF,false,car,null
1,AUDI,RS6,RS6 Avant 5.0 V10 quattro,Zuzwil,7,2008,25400,Occasion,Combi,anthracite,CH,l_km_consumption,kilometer,LHD,CHF,false,car,null
2,AUDI,RS6,RS6 Avant quattro,Zuzwil,10,2002,38500,Occasion,Combi,anthracite,CH,l_km_consumption,kilometer,LHD,CHF,false,car,null


_formatting make, color and mileage as required_

In [46]:
norm2.make = norm2.make.apply(lambda str : str.title())
norm2.color = norm2.color.apply(lambda str : str.title())
norm2.mileage = norm2.mileage.apply(mileage_conv)

In [47]:
norm3 = norm2.copy()

_providing new order for columns in order to match target data_

In [48]:
cols = [
    'carType',
    'color',
    'condition',
    'currency',
    'drive',
    'city',
    'country',
    'make',
    'manufacture_year',
    'mileage',
    'mileage_unit',
    'model',
    'model_variant',
    'price_on_request',
    'type',
    'zip',
    'manufacture_month',
    'fuel_consumption_unit'
]

norm3 = norm3.reindex(columns=cols)

In [54]:
norm3.shape

(1153, 18)

_saving data to existing excel template_

In [50]:
book = load_workbook('rawdata/extemp.xlsx')
writer = pd.ExcelWriter('rawdata/extemp.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

ex.to_excel(writer, "preprocessing", index=False)
norm3.to_excel(writer, "normalisation", index=False)
norm3.to_excel(writer, "integration", index=False, header=False, startrow=writer.sheets['integration'].max_row)

for ws in writer.sheets.values():
    removeFormatting(ws)

writer.save()